# TESScut and ExoMAST: Working with TESS Time Series Data

In this tutorial we will learn about MAST's programmatic tools for accessing TESS time series data while exploring a "weird looking" light curve.  We will follow up on unusual TCE results using the MAST API in Python to access and view TESS time series and FFI data.

Topics to be covered include:
- Using the MAST API to get data validation time series
- Plotting TESS light curves in Python
- Using the MAST API to make an FFI cutout
- Creating a movie of TPF frames in Python

See the __[MAST TESS site](http://archive.stsci.edu/tess/)__ for more information and example on how to access and use TESS data.

## Terminology

- **TESS:** The Transiting Exoplanet Survey Satellite
- **Sector:** TESS observed the sky in regions of 24x96 degrees along the southern, then northern, ecliptic hemispheres. Each of these regions is referred to as a "sector", starting with Sector 1.
- **TCE:** Threshold Crossing Event, periodic signals found by the TESS pipeline that exceed a nominal signal-to-noise ratio.
- **Data Validation (DV):** The Data Validation (DV) module of the pipeline produces a set of products that can help validate the quality of a TCE. The DV products include a time series file of the flattened light curve that was searched and relevant statistics for each signal (dvt.fits), DV reports that consists of a few diagnostic plots and relevant statistics (dvs.pdf for individual signals, dvr.pdf for all signals found in the TIC object), and an xml file (dvr.xml) that contains the results of the planet transit fit. We will be exploring a dvt.fits file in this tutorial.
- **FFI:** TESS periodically reads out the entire frame of all four cameras, nominally every 30 minutes, and stores them as full frame images (FFIs). 
- **HDU:** Header Data Unit. A FITS file is made up of HDUs that contain data and metadata relating to the file. The first HDU is called the primary HDU, and anything that follows is considered an "extension", e.g., "the first FITS extension", "the second FITS extension", etc.
- **HDUList:** A list of HDUs that comprise a fits file.
- **BJD:** Barycentric Julian Date, the Julian Date that has been corrected for differences in the Earth's position with respect to the Solar System center of mass.
- **BTJD:** Barycentric TESS Julian Date, the timestamp measured in BJD, but offset by 2457000.0. I.e., BTJD = BJD - 2457000.0

## Imports

In this tutorial we will use a HTTP based web service API, as well as Astroquery to find and download data.

We will use both the matplotlib and bokeh packages to visualize our data as they have different strengths and weaknesses.

In [ ]:
# For querying for data
import requests
from astroquery.mast import Tesscut

# For manipulating data
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord

import re

# For matplotlib plotting
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# For animation display
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')

# For bokeh plotting
from bokeh import plotting
plotting.output_notebook()

## The Problem

There is something weird in the lightcurve of a particular TESS object, we want to find out what is causing the weirdness.  We know the TIC ID of the object in question.

In [ ]:
weird_tic_id = "00214568914"

## Using requests to query ExoMAST

__[ExoMAST](https://exo.mast.stsci.edu)__ is MAST's exoplanet specific interface.  We can access its functionality programmatically by using its __[HTTP based API](https://exo.mast.stsci.edu/docs/)__.

The main ExoMAST API url is `https://exo.mast.stsci.edu/api/v0.1/`.
We will use several queries in this tutorial:
- Find TCEs associated with a given TIC ID: `dvdata/tess/<TIC ID>/tces/`
- Get DV metadata associated with a particular TCE: `dvdata/tess/<TIC ID>/info/?tce=<TCE>`
- Get the DV time series associated with a particular TCE: `dvdata/tess/<TIC ID>/table/?tce=<TCE>`

In [ ]:
exomast_url = "https://exo.mast.stsci.edu/api/v0.1/"

### Listing the TCEs associated with our TIC ID

To do this we build the query url and then perform an HTTP GET using the `requests` package.

In [ ]:
list_tce_query = f"{exomast_url}dvdata/tess/{weird_tic_id}/tces/"
list_tce_query

In [ ]:
response = requests.get(list_tce_query)
tce_dict = response.json()
print(tce_dict)

In [ ]:
weird_tce = tce_dict['TCE'][0]

### Getting the meta data associated with our TCE
To do this we build the metadata query url and then perform another HTTP GET.

In [ ]:
dv_metadata_query = f"{exomast_url}dvdata/tess/{weird_tic_id}/info/?tce={weird_tce}"
dv_metadata_query

In [ ]:
response = requests.get(dv_metadata_query)
metadata = response.json()
metadata.keys()

In [ ]:
metadata['DV Primary Header'].keys()

In [ ]:
metadata['DV Data Header'].keys()

## Getting the DV data time series associated with our TCE

This time we build a data table query url and then perform the HTTP GET.

In [ ]:
dv_lightcurve_query = f"{exomast_url}dvdata/tess/{weird_tic_id}/table/?tce={weird_tce}"
dv_lightcurve_query

In [ ]:
response = requests.get(dv_lightcurve_query)
data_dict = response.json()
data_dict.keys()

In [ ]:
for col in data_dict['fields']:
    print(f"{col['colname']} {col['datatype']:5} {col['description']}")

### Turning our JSON data into an Astropy Table

This step is not strictly necessary, we could certainly use our data directly in its current form, however, it is often convenient to put it in a data table.  Here we will use a function to put our results into an Astropy table, but you could similarly fill a Pandas table or numpy array.

In [ ]:
def json_to_table(fields, data):
    """"
    Takes a json object and turns it into an astropy table.

    Parameters
    ----------
    fields : list of dicts
        Of the form [{colname:,datatype:,description:}, ...]
    data : list of dicts
       Of the form [{col1:, col2:, ...},{col1:, col2:, ...}, ...]

    Returns
    -------
    response : `astropy.table.Table`
    """

    rx = re.compile(r"varchar\((\d+)\)")
    
    data_table = Table()

    for col, atype in [(x['colname'], x['datatype']) for x in fields]:
        col = col.strip()
        if "varchar" in atype:
            match = rx.search(atype)
            atype = "U" + match.group(1)
        if atype == "real":
            atype = "float"
        data_table[col] = np.array([x.get(col, None) for x in data], dtype=atype)

    return data_table


In [ ]:
weird_lightcurve = json_to_table(data_dict['fields'],data_dict['data'])

In [ ]:
weird_lightcurve

## Exploring our light curve

We will start by using `matplotlib` to quickly plot up the detrended light curve by phase, which will allow us to observe TCE signal.

In [ ]:
fig, ax = plt.subplots(figsize=(14,4))

# plotting the phased light curve
ax.plot(weird_lightcurve["PHASE"], weird_lightcurve['LC_DETREND'], 
        marker='.',linestyle='None', markersize=1, markerfacecolor='black', markeredgecolor="black") 
    
plt.title(f"Detrended Lightcurve (TIC{weird_tic_id}-{weird_tce})")            
plt.show(block=False)

There is definitely *something* happening in that light curve.  We will now use `bokeh` so we can zoom in on the anomaly.

In [ ]:
bfig = plotting.figure(plot_width=850, plot_height=250, title=f"Detrended Lightcurve (TIC{weird_tic_id})")
bfig.circle(weird_lightcurve["PHASE"],weird_lightcurve["LC_DETREND"], fill_color="black",size=1, line_color=None)
plotting.show(bfig)

## Making an FFI cutout

We want to see what is going on with this light curve, so we will make a cutout around the object accross the entire sectore, and then make a movie that shows how it changes over time.

We will use the `astroquery.mast` __[Tesscut](https://astroquery.readthedocs.io/en/latest/mast/mast.html#tesscut)__ class to make this cutout.  
We will use two functions:
- Find the sectors in which our object was observed: `Tesscut.get_sectors`
- Query for cutouts and get the result as a list of HDUList objects: `Tesscut.get_cutouts`

These queries require us to know the RA and Dec of our object of interest, which was in the metadata we queried earlier in the tutorial. 

In [ ]:
ra = metadata['DV Data Header']['RA_OBJ']
dec = metadata['DV Data Header']['DEC_OBJ']
obj_coord = SkyCoord(ra,dec,unit="deg")
print(obj_coord)

**Getting a list of TESS sectors that observed our target.**

In [ ]:
Tesscut.get_sectors(obj_coord)

**Requesting a cutout target pixel file.**

This query will return a list of `HDUList` objects, each of which is the cutout target pixel file for a single sector. In this case, because we did a sector query and know that our target only appears in one sector, we know that the resulting list will only have one element and can pull it out directly.

In [ ]:
cutout_hdu = Tesscut.get_cutouts(obj_coord, size=50)[0]

In [ ]:
cutout_hdu.info()

In [ ]:
cutout_table = cutout_hdu[1].data
cutout_table.columns

## Exploring the cutout time series

We want to explore what is happening with in our cutout area over time, so we will make an animated plot of the cutout frames.

### Deciding which timeframe to look at

We can't make a movie of the whole sector (it would take too long), so we will look at the light curve to see where we think the weirdness happens.

This time instead of plotting the phased light curve, we will just plot the detrended lightcurve against time (in BTJD).

In [ ]:
bfig = plotting.figure(plot_width=850, plot_height=250, title=f"Detrended Lightcurve (TIC{weird_tic_id})")
bfig.circle(weird_lightcurve["TIME"],weird_lightcurve["LC_DETREND"], fill_color="black",size=1, line_color=None)
plotting.show(bfig)

Here we can see which parts we want to explore in more depth, we just need to figure out which frames of the cutout array correspond to the timestamps in our plot.

In [ ]:
def find_index(btjd):
    """
    Given a time as a Barycentric TESS Julian Date (BTJD) timestamp, return the closest index in a table
    that is assumed to have a TIME column that is also in BTJD"""
    
    return (np.abs(cutout_table['TIME'] - btjd)).argmin()

In [ ]:
start = find_index(1334)
end = find_index(1335)

print(f"Frames {start}-{end} ({end-start} frames)")

### Looking at the animated cutout

In [ ]:
def make_animation(data_array, start_frame=0, end_frame=None, vmin=None, vmax=None, delay=50):
    """
    Function that takes an array where each frame is a 2D image array and make an animated plot
    that runs through the frames.
    
    Note: This can take a long time to run if you have a lot of frames.    
    Parameters
    ----------
    data_array : array
        Array of 2D images.
    start_frame : int
        The index of the initial frame to show. Default is the first frame.
    end_frame : int
        The index of the final frame to show. Default is the last frame.
    vmin : float
        Data range min for the colormap. Defaults to data minimum value.
    vmax : float
        Data range max for the colormap. Defaults to data maximum value.
    delay: 
        Delay before the next frame is shown in milliseconds.

    Returns
    -------
    response : `animation.FuncAnimation`
    """
    
    if not vmin:
        vmin = np.min(data_array)
    if not vmax:
        vmax = np.max(data_array)
        
    if not end_frame:
        end_frame = len(data_array) - 1 # set to the end of the array
        
    num_frames = end_frame - start_frame + 1 # include the end frame
        
    def animate(i, fig, ax, binarytab, start=0):
        """Function used to update the animation"""
        ax.set_title("Epoch #" + str(i+start))
        im = ax.imshow(binarytab[i+start], cmap=plt.cm.YlGnBu_r, vmin=vmin, vmax=vmax)
        return im,
    
    # Create initial plot.
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(data_array[start_frame], cmap=plt.cm.YlGnBu_r, vmin=vmin, vmax=vmax)

    ani = animation.FuncAnimation(fig, animate, fargs=(fig, ax, data_array, start_frame), frames=num_frames, 
                                  interval=delay, repeat_delay=1000)
    
    plt.close()
    
    return ani

In [ ]:
make_animation(cutout_table['FLUX'], start, end, vmax=500)